In [ ]:
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=c7429d5f069f1fabee32e2ae27d457db9b843404ec21343dbc01be31400828a7
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
import pickle
import json
#import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
import numpy as np
import re
import os

from unidecode import unidecode
from collections import Counter
from math import ceil
from langdetect import detect
from random import sample

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
rutaDatos = '../Datos'
ruta = './'

## Support Files

Throughout the modeling process, some of the model artifacts needed to be updated and so this notebook was used to quickly update those files.

In [4]:
# location where current files are located
curr_model_artifacts_location = rutaDatos + '/institution_tagger_v2_artifacts' +'/'

# Load the needed files
with open(f"{curr_model_artifacts_location}departments_list.pkl", "rb") as f:
    departments_list = pickle.load(f)

print("Loaded list of departments")

with open(f"{curr_model_artifacts_location}countries_list_flat.pkl", "rb") as f:
    countries_list_flat = pickle.load(f)

print("Loaded flat list of countries")

with open(f"{curr_model_artifacts_location}countries.json", "r") as f:
    countries_dict = json.load(f)

print("Loaded countries dictionary")

with open(f"{curr_model_artifacts_location}city_country_list.pkl", "rb") as f:
    city_country_list = pickle.load(f)

print("Loaded strings of city/country combinations")

Loaded list of departments
Loaded flat list of countries
Loaded countries dictionary
Loaded strings of city/country combinations


In [5]:
print(departments_list)
print(countries_list_flat)
print(city_country_list)
countries_dict

['Mechanical and Aerospace Engineering', 'Political Science', 'Agriculture', 'Foreign Languages and Literatures', 'Biomedical Engineering', 'Nephrology', 'Literature and Creative Writing', 'Interdisciplinary Studies', 'Physics', 'Mathematics', 'Engineering', 'Civil, Construction and Environmental Engineering', 'Thoracic and Cardiovascular Surgery', 'English', 'Other Departments', 'Internal Medicine', 'Communication', 'Electrical Engineering', 'Psychiatry', 'Environmental Engineering', 'Industrial & Systems Engineering', 'Statistics', 'Social Work', 'Philosophy and Religious Studies', 'Public Administration', 'Materials Science and Engineering', 'Medicine', 'IT', 'Civil Engineering', 'Neuroscience', 'Biological Sciences', 'Psychology', 'Sociology and Anthropology', 'Nursing & Midwifery', 'Surgery', 'Other departments', 'Biological and Agricultural Engineering', 'Chemistry', 'Applied Mechanics', 'History', 'Aerospace Engineering', 'Nuclear Engineering', 'Computer Science', 'Information T

{'Andorra': ['Andorra'],
 'United Arab Emirates': ['United Arab Emirates'],
 'Afghanistan': ['Afghanistan'],
 'Antigua and Barbuda': ['Antigua and Barbuda'],
 'Anguilla': ['Anguilla'],
 'Albania': ['Albania'],
 'Armenia': ['Armenia'],
 'Angola': ['Angola'],
 'Antarctica': ['Antarctica'],
 'Argentina': ['Argentina'],
 'American Samoa': ['American Samoa'],
 'Austria': ['Austria'],
 'Australia': ['Australia'],
 'Aruba': ['Aruba'],
 'Aland Islands': ['Åland Islands', 'Aland Islands'],
 'Azerbaijan': ['Azerbaijan'],
 'Bosnia and Herzegovina': ['Bosnia and Herzegovina'],
 'Barbados': ['Barbados'],
 'Bangladesh': ['Bangladesh'],
 'Belgium': ['Belgium'],
 'Burkina Faso': ['Burkina Faso'],
 'Bulgaria': ['Bulgaria'],
 'Bahrain': ['Bahrain'],
 'Burundi': ['Burundi'],
 'Benin': ['Benin'],
 'Saint Barthélemy': ['Saint Barthélemy'],
 'Bermuda': ['Bermuda'],
 'Brunei': ['Brunei'],
 'Bolivia': ['Bolivia (Plurinational State of)', 'Bolivia'],
 'Bonaire, Saint Eustatius and Saba ': ['Bonaire, Sint Eusta

### Looking at ROR

In [6]:
def get_geoname_admin(address_dict):
    try:
        geoname_admin = address_dict['geonames_city']['geonames_admin1']['name']
    except:
        geoname_admin = "None"

    return geoname_admin

In [7]:
def get_final_region(ror_state, ror_region):
    if isinstance(ror_state, str):
        return ror_state
    elif isinstance(ror_region, str):
        return ror_region
    else:
        return None

In [8]:
def check_for_backwards_univ(curr_names):
    names = curr_names.copy()
    for one_name in curr_names:
        split_name = one_name.split(" ")
        if len(split_name) == 3:
            if (split_name[0] == 'University') & (split_name[1] == 'of'):
                names.append(f"{split_name[2]} University")
        elif len(split_name) == 2:
            if (split_name[1] == 'University'):
                names.append(f"University of {split_name[0]}")
        else:
            pass
    return names

In [9]:
def add_names_to_list(all_names):
    names = all_names.copy()
    if "Harvard University" in names:
        names.append("Harvard Medical School")
    elif "University of Oxford" in names:
        names.append("Oxford University")
    else:
        pass

    return names

In [10]:
def get_exact_names(name, aliases, acronyms, labels):
    all_names = [name] + aliases + acronyms + [i['label'] for i in labels]
    all_names = add_names_to_list(all_names)
    all_names = [x for x in all_names if not(x.startswith('Department of'))]  #cambio operador
    all_names_clean = [string_match_clean(x) for x in all_names]
    return [x for x in all_names_clean if len(x) > 4]

In [11]:
def string_match_clean(text):
    #replace "&" with "and"
    if "r&d" not in text.lower():
        text = text.replace(" & ", " and ")

    # take country out
    if text.strip().endswith(")"):
        for country in countries_list_flat:
            if text.strip().endswith(f"({country})"):
                text = text.replace(f"({country})", "")

    # use unidecode
    text = unidecode(text.strip())

    # replacing common abbreviations
    text = text.replace("Univ.", "University")
    text = text.replace("Lab.", "Laboratory")

    # take out spaces, commas, dashes, periods, etcs
    text = re.sub("[^0-9a-zA-Z]", "", text)

    return text

In [12]:
def list_of_all_names(oa_name, ror_names, extra_names, use_extra_names=False):
    banned_names = ['UniversityHospital','Coastal','Brunswick','Continental']
    if isinstance(ror_names, list):
        pass
    else:
        ror_names = []

    if isinstance(oa_name, str):
        oa_string = [string_match_clean(oa_name)]
    else:
        oa_string = []

    if use_extra_names:
        if isinstance(extra_names, list):
            pass
        else:
            extra_names = []
    else:
        extra_names = []

    return [x for x in list(set(oa_string+ror_names+extra_names)) if
            ((len(x) > 4) & (x not in banned_names))]

In [13]:
with open(f"{rutaDatos}/institOA-AR-P1.txt","r") as archivo1:
    instText1 =archivo1.read()
with open(f"{rutaDatos}/institOA-AR-P2.txt","r") as archivo2:
    instText2 =archivo2.read()

instDict1 = json.loads(instText1)
instDict2 = json.loads(instText2)

instList1 = instDict1['results']
instList2 = instDict2['results']

instTotal = instList1 + instList2
instDF = pd.DataFrame.from_dict(instTotal)
instDF = instDF.rename(columns={"id": "affiliation_id", "ror": "ror_id"})
instDF['ror_id'] = instDF['ror_id'].apply(lambda x: x.split("/")[-1])
instDF['affiliation_id'] = instDF['affiliation_id'].apply(lambda x: x.split("/")[-1])
instDF.to_parquet(f"{rutaDatos}/OA_static_institutions_single_file.parquet")

In [14]:
# this file is not provided but the needed data is all institutions in OpenAlex
# with the following columns: 'ror_id','affiliation_id','display_name'
institutions_df = pd.read_parquet(f"{rutaDatos}/OA_static_institutions_single_file.parquet")

In [15]:
print(institutions_df.columns)
print(institutions_df.shape)
institutions_df = institutions_df.get(['affiliation_id', 'ror_id', 'display_name'])
institutions_df['ror_id'] = institutions_df['ror_id'].apply(lambda x: x.split("/")[-1])
institutions_df

Index(['affiliation_id', 'ror_id', 'display_name', 'country_code', 'type',
       'type_id', 'lineage', 'homepage_url', 'image_url',
       'image_thumbnail_url', 'display_name_acronyms',
       'display_name_alternatives', 'repositories', 'works_count',
       'cited_by_count', 'summary_stats', 'ids', 'geo', 'international',
       'associated_institutions', 'counts_by_year', 'roles', 'topics',
       'topic_share', 'x_concepts', 'is_super_system', 'works_api_url',
       'updated_date', 'created_date'],
      dtype='object')
(331, 29)


,affiliation_id,ror_id,display_name
0,I151201029,03cqe8w59,Consejo Nacional de Investigaciones Científicas y Técnicas
1,I24354313,0081fs513,University of Buenos Aires
2,I874386039,01tjs6929,Universidad Nacional de La Plata
3,I166401450,056tb7j80,Universidad Nacional de Córdoba
4,I53241121,05rxmkq09,Fundación Ciencias Exactas y Naturales
...,...,...,...
326,I4210128047,02by8jy67,Fundación TyPA
327,I4210131417,03qryrx52,Argentine Atlantis University
328,I4210153804,04vc7kq70,Council of Magistrates of the Nation
329,I4210158077,05pqpm243,Junior Achievement


In [16]:
# institutions = institutions_df.set_index('affiliation_id').to_dict(orient='index')

In [17]:
ror = pd.read_json(f"{rutaDatos}/v1.50-2024-07-29-ror-data.json")
ror = ror[ror.country.apply(lambda x: x['country_code'] == 'AR')]
ror.shape

(388, 16)

In [18]:
type(ror.loc[213]['name'])  #str
ror.loc[213]['name']

'National University of the Littoral'

In [19]:
ror['address'] = ror['addresses'].apply(lambda x: x[0])
ror['ror_id'] = ror['id'].apply(lambda x: x.split("/")[-1])
ror['types'] = ror['types'].apply(lambda x: x[0])

In [20]:
ror.head(1)

,id,name,types,links,aliases,acronyms,status,wikipedia_url,labels,email_address,ip_addresses,established,country,relationships,addresses,external_ids,address,ror_id
213,https://ror.org/00pt8r998,National University of the Littoral,Education,[http://www.unl.edu.ar/],[],[UNL],active,http://en.wikipedia.org/wiki/National_University_of_the_Littoral,"[{'label': 'Universidad Nacional del Litoral', 'iso639': 'es'}, {'label': 'Université Nationale du Littoral', 'iso639': 'fr'}]",,[],1889.0,"{'country_code': 'AR', 'country_name': 'Argentina'}","[{'type': 'Child', 'label': 'Centro de Investigación de Métodos Computacionales', 'id': 'https://ror.org/0041aya12'}, {'type': 'Child', 'label': 'Instituto Nacional de Limnología', 'id': 'https://ror.org/042765079'}, {'type': 'Child', 'label': 'Instituto de Agrobiotecnología del Litoral', 'id': 'https://ror.org/057915t59'}, {'type': 'Child', 'label': 'Instituto de Desarrollo Tecnológico para la Industria Química', 'id': 'https://ror.org/01kwwh635'}, {'type': 'Child', 'label': 'Instituto de Física del Litoral', 'id': 'https://ror.org/058xqms97'}]","[{'line': None, 'lat': -31.634444, 'lng': -60.705278, 'postcode': None, 'primary': False, 'city': 'Santa Fe', 'state': None, 'state_code': None, 'country_geonames_id': 3865483, 'geonames_city': {'id': 3836277, 'city': 'Santa Fe', 'nuts_level1': {'code': None, 'name': None}, 'nuts_level2': {'code': None, 'name': None}, 'nuts_level3': {'code': None, 'name': None}, 'geonames_admin1': {'id': 3836276, 'name': 'Santa Fe', 'ascii_name': 'Santa Fe', 'code': 'AR.21'}, 'geonames_admin2': {'id': 3852292, 'name': 'Departamento de La Capital', 'ascii_name': 'Departamento de La Capital', 'code': 'AR.21.82063'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}}}]","{'ISNI': {'preferred': None, 'all': ['0000 0001 2172 9456']}, 'FundRef': {'preferred': None, 'all': ['501100005746']}, 'OrgRef': {'preferred': None, 'all': ['7447908']}, 'Wikidata': {'preferred': None, 'all': ['Q265866']}, 'GRID': {'preferred': 'grid.10798.37', 'all': 'grid.10798.37'}}","{'line': None, 'lat': -31.634444, 'lng': -60.705278, 'postcode': None, 'primary': False, 'city': 'Santa Fe', 'state': None, 'state_code': None, 'country_geonames_id': 3865483, 'geonames_city': {'id': 3836277, 'city': 'Santa Fe', 'nuts_level1': {'code': None, 'name': None}, 'nuts_level2': {'code': None, 'name': None}, 'nuts_level3': {'code': None, 'name': None}, 'geonames_admin1': {'id': 3836276, 'name': 'Santa Fe', 'ascii_name': 'Santa Fe', 'code': 'AR.21'}, 'geonames_admin2': {'id': 3852292, 'name': 'Departamento de La Capital', 'ascii_name': 'Departamento de La Capital', 'code': 'AR.21.82063'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}}}",00pt8r998


In [21]:
ror['country_code'] = ror['country'].apply(lambda x: x['country_code'])
ror['country_name'] = ror['country'].apply(lambda x: x['country_name'])
ror['city'] = ror['address'].apply(lambda x: x['city'])
ror['state'] = ror['address'].apply(lambda x: x['state'])
ror['region'] = ror['address'].apply(get_geoname_admin)

In [22]:
ror_to_join = ror[['ror_id','name','status','types','aliases','acronyms','labels','city',
                   'state','region','country_name']].copy()

ror_to_join.columns = ['ror_id','name','status','types','aliases','acronyms','labels','city',
                       'temp_state','temp_region','country']

In [23]:
ror_to_join['state'] = ror_to_join.apply(lambda x: get_final_region(x.temp_state, x.temp_region), axis=1)

In [24]:
ror_to_join.head(1)

,ror_id,name,status,types,aliases,acronyms,labels,city,temp_state,temp_region,country,state
213,00pt8r998,National University of the Littoral,active,Education,[],[UNL],"[{'label': 'Universidad Nacional del Litoral', 'iso639': 'es'}, {'label': 'Université Nationale du Littoral', 'iso639': 'fr'}]",Santa Fe,None,Santa Fe,Argentina,Santa Fe


In [25]:
inst_ror = ror_to_join.merge(institutions_df[['ror_id','affiliation_id','display_name']],
                             how='inner', on='ror_id')
print(inst_ror.shape)
inst_ror.head(1)

(331, 14)


,ror_id,name,status,types,aliases,acronyms,labels,city,temp_state,temp_region,country,state,affiliation_id,display_name
0,00pt8r998,National University of the Littoral,active,Education,[],[UNL],"[{'label': 'Universidad Nacional del Litoral', 'iso639': 'es'}, {'label': 'Université Nationale du Littoral', 'iso639': 'fr'}]",Santa Fe,None,Santa Fe,Argentina,Santa Fe,I3121987637,National University of the Littoral


In [26]:
print(ror_to_join[['ror_id','name']].head(1))
print(institutions_df[['ror_id','affiliation_id']].head(1))

        ror_id                                 name
213  00pt8r998  National University of the Littoral
      ror_id affiliation_id
0  03cqe8w59     I151201029


#### Getting file of multi-institution names

In [27]:
ror_to_join[['ror_id','name']].merge(institutions_df[['ror_id','affiliation_id']],how='left', on='ror_id')['name'].value_counts().index

Index(['National University of the Littoral',
       'National Center for Genome Resources',
       'Instituto de Investigaciones Psicológicas',
       'Fundación Salud, Ciencia y Tecnologia',
       'National University of Rosario',
       'Instituto de Investigación Médica Mercedes y Martín Ferreyra',
       'Universidad Nacional Arturo Jauretche',
       'Colegio Nacional de Monserrat', 'Universidad Provincial de Córdoba',
       'Research Center on Earth Sciences',
       ...
       'Instituto de Física del Litoral', 'Institute of Physics Rosario',
       'Wildlife Conservation Society Argentina', 'Junior Achievement',
       'Chemotecnica (Argentina)',
       'Instituto Argentino de Diagnóstico y Tratamiento',
       'Biblioteca del Congreso de la Nación',
       'Servicio Geológico Minero Argentino',
       'Center for Studies on Sustainable Luxury',
       'Universidad Nacional de Córdoba'],
      dtype='object', name='name', length=388)

In [28]:
multi_inst_names_df = ror_to_join[['ror_id','name']].merge(institutions_df[['ror_id','affiliation_id']],how='left', on='ror_id')['name'].value_counts().reset_index()

In [29]:
print(multi_inst_names_df.columns)
multi_inst_names_df.head(2)

Index(['name', 'count'], dtype='object')


,name,count
0,National University of the Littoral,1
1,National Center for Genome Resources,1


In [30]:
multi_inst_names_df['count']>1

0      False
1      False
2      False
3      False
4      False
       ...  
383    False
384    False
385    False
386    False
387    False
Name: count, Length: 388, dtype: bool

In [31]:
multi_inst_names_df[multi_inst_names_df['count']>1]

,name,count


In [32]:
multi_inst_names = multi_inst_names_df[multi_inst_names_df['count']>1].iloc[:,0].tolist()  #inst
multi_inst_names

[]

In [33]:
multi_inst_names_ids = inst_ror[inst_ror['name'].isin(multi_inst_names)]['affiliation_id'].tolist()

In [34]:
with open(f"{curr_model_artifacts_location}multi_inst_names_ids.pkl", "wb") as f:   #instituciones con mutiples name. Lita vacia en este caso
  pickle.dump(multi_inst_names_ids, f)

### Getting Mapping of Inactive Institutions

There are institutions in ROR that are listed as "Withdrawn" or "Inactive". There was some thought to use the old data associated with these ROR IDs and apply them to successors but for this model, we decided to hold off on doing this because we were unsure if there would be a benefit to doing so. Therefore, the code is provided but this data was not used in building the model.

In [35]:
def get_successors_from_relationships(relationships):
    successors = []
    parents = []
    for relationship in relationships:
        if relationship['type'] == 'Successor':
            successors.append(relationship['id'].split("/")[-1])
        elif relationship['type'] == 'Parent':
            parents.append(relationship['id'].split("/")[-1])
        else:
            pass
    return [successors, parents]

In [36]:
def get_extra_names(ror_id):
    if ror_id in successor_dict.keys():
        extra_names = []
        for old_id in successor_dict[ror_id]['ror_id']:
            extra_names += old_name_data[old_id]['successor_names']

        extra_names = list(set(extra_names))
    else:
        extra_names = []

    return extra_names

In [37]:
withdrawn_or_inactive_df = ror[ror['status'].isin(['withdrawn','inactive'])].copy()
withdrawn_or_inactive_df.shape

(3, 23)

In [38]:
withdrawn_or_inactive_df['successors_parents'] = withdrawn_or_inactive_df['relationships'] \
    .apply(get_successors_from_relationships)

In [39]:
withdrawn_or_inactive_df['successors'] = withdrawn_or_inactive_df['successors_parents'].apply(lambda x: x[0])
withdrawn_or_inactive_df['parents'] = withdrawn_or_inactive_df['successors_parents'].apply(lambda x: x[1])

In [40]:
withdrawn_or_inactive_df['successor_len'] = withdrawn_or_inactive_df['successors'].apply(len)

In [41]:
to_add_to_successors = withdrawn_or_inactive_df[withdrawn_or_inactive_df['successor_len']==1].copy()
to_add_to_successors['successor'] = to_add_to_successors['successors'].apply(lambda x: x[0])
to_add_to_successors.shape

(3, 28)

In [42]:
to_add_to_successors['successor_names'] = to_add_to_successors.apply(lambda x: get_exact_names(x['name'],
                                                                                               x.aliases,
                                                                                               x.acronyms,
                                                                                               x.labels),
                                                                     axis=1)

In [43]:
old_name_data = to_add_to_successors.set_index('ror_id')[['successor_names']].to_dict(orient='index')

In [44]:
successor_dict = to_add_to_successors.groupby('successor')['ror_id'].apply(list).reset_index()\
    .set_index('successor').to_dict(orient='index')

### Getting ROR String Matching File and Affiliation Dictionary

In [45]:
inst_ror['extra_names'] = inst_ror['ror_id'].apply(get_extra_names)

In [46]:
inst_ror.columns

Index(['ror_id', 'name', 'status', 'types', 'aliases', 'acronyms', 'labels',
       'city', 'temp_state', 'temp_region', 'country', 'state',
       'affiliation_id', 'display_name', 'extra_names'],
      dtype='object')

In [47]:
inst_ror['exact_names'] = inst_ror.apply(lambda x: get_exact_names(x['name'], x.aliases,
                                                                         x.acronyms, x.labels), axis=1)

In [48]:
inst_ror['final_names'] = inst_ror.apply(lambda x: list_of_all_names(x.display_name, x.exact_names,
                                                                     x.extra_names,
                                                                     use_extra_names=False), axis=1)

In [49]:
new_affiliation_dict = inst_ror.set_index('affiliation_id')[['display_name','city','state',
                                                             'country','final_names','ror_id','types']] \
.to_dict(orient='index')

In [50]:
with open(f"{curr_model_artifacts_location}full_affiliation_dict.pkl", "wb") as f:
    pickle.dump(new_affiliation_dict, f)

### Updating the city/country file

This file is used to check the affiliation string to make sure it doesn't exactly match up with a city/region/country combo with no additional information.

In [51]:
city_region_country = inst_ror.drop_duplicates(subset=['city','country']).copy()
city_region_country.shape

(59, 17)

In [52]:
new_city_country_list = list(set([f"{i}{j}" for i,j in zip(city_region_country['city'].tolist(),
                                   city_region_country['country'].tolist())] +
         [f"{i}{j}{k}"for i,j,k in zip(city_region_country['city'].tolist(),
                                             city_region_country['state'].tolist(),
                                             city_region_country['country'].tolist()) if j ] +
         [f"{i}{j}" for i,j in zip(city_region_country['state'].tolist(),
                                   city_region_country['country'].tolist()) if i] +
         [f"{i}" for i in city_region_country['country'].tolist()] +
         [f"{i}" for i in city_region_country['state'].tolist() if i]))

new_city_country_list = list(set([string_match_clean(x) for x in new_city_country_list]))

In [53]:
len(new_city_country_list)

149

In [54]:
with open(f"{curr_model_artifacts_location}city_country_list.pkl", "wb") as f:
    pickle.dump(new_city_country_list, f)

### Flat country file is up to date

Flat country file is needed to search for country in the string for the model.

In [55]:
len(list(set(countries_list_flat)))

396

In [56]:
all_countries = []
for i in countries_dict.values():
    all_countries += i

In [57]:
len(list(set(all_countries)))

396

In [58]:
with open(f"{curr_model_artifacts_location}countries_list_flat.pkl", "wb") as f:
    pickle.dump(list(set(all_countries)), f)

### Departments list update

Takes the old department list and updates it with additional department names.

In [59]:
with open(f"{ruta}/ror_string_beginnings/Education_dept", 'r') as f:
    education_dept_begs = f.readlines()

education_dept_begs = list(set([x.rstrip('\n') for x in education_dept_begs]))

In [60]:
departments_list = ['Psychology','Nephrology','Other departments','Other Departments','Nursing & Midwifery',
                    'Literature and Creative Writing','Neuroscience','Engineering','Computer Science',
                    'Chemistry','Biology','Medicine']

In [61]:
new_departments_list = list(set(departments_list + education_dept_begs))

In [62]:
with open(f"{curr_model_artifacts_location}departments_list.pkl", "wb") as f:
    pickle.dump(new_departments_list, f)

### Make affiliation IDs integers

In [63]:
with open(f"{curr_model_artifacts_location}affiliation_vocab.pkl", "rb") as f:
    affiliation_vocab_basic = pickle.load(f)

new_affiliation_vocab_basic = {int(i):int(j) for j,i in affiliation_vocab_basic.items()}

print("Loaded basic affiliation vocab")

Loaded basic affiliation vocab


In [64]:
with open(f"{curr_model_artifacts_location}affiliation_vocab.pkl", "wb") as f:
    pickle.dump(new_affiliation_vocab_basic, f)